In [1]:
import numpy as np
import os, glob
import gdal, osr # python2.7
# import rasterio # python2.7
# from osgeo import gdal
import MisrToolkit as Mtk # python2.7
from MisrToolkit import * # 

## my assumptions

In [2]:
path_num = 180
misr_img_res = (512, 2048)
# img_nrows = misr_img_dims[0]
# img_ncols = misr_img_dims[1]
misr_res_meter = 275

## My functions in this code

### file path checker

In [3]:
def file_fullpath_checker(file_fulPath):
    "this function checks if "
    if not os.path.isfile(file_fulPath):
        print("-> ERROR: file or directory NOT available!")
        print(file_fulPath)
    else:
        print("-> file available!")
    return 0

### raster file name

In [4]:
def setup_rastername(output_filename, raster_dir_fullpath):
    "setup directory and file name for output raster"
    #~ set raster label 
    raster_label = "LatLon_degreeRes_NorthUp_perspectiveTrans"
    #~ define output raster
    raster_name = output_filename+'_'+raster_label+'.tif'
    # print(raster_name)
    #~ define raster output dir
    raster_fullPath = os.path.join(raster_dir_fullpath, raster_name)
    print("-> output raster: %s" % (raster_name))
    # print("-> output raster fullPath:")
    print(raster_fullPath)
    return raster_fullPath

### swap y-coords of SMO(x,y)

In [5]:
# def swap_ul_lr_y_coord(ulc_tuple, lrc_tuple):
#     "we need this for SOMxy if we are using SOM"
#     print('-> corner-elements swap mode: "swap_ul_lr_y_coord"')
#     #~ use top left corner or each block to define corners of block1
#     #~ We swap them based on MISR docs, values are in SOM projection
#     ulc_somx = ulc_tuple[0] # somx
#     ulc_somy = lrc_tuple[1] # tl somy is swapped with brc somy

#     lrc_somx = lrc_tuple[0] # as lon
#     lrc_somy = ulc_tuple[1] # as lat

#     ulc_som = (ulc_somx, ulc_somy)
#     lrc_som = (lrc_somx, lrc_somy)
    
    
#     print("-> ulc y-swapped= %s, %s " %(ulc_somx, ulc_somy))
#     print("-> lrc y-swapped= %s, %s " %(lrc_somx, lrc_somy))
#     # proj_param_res = proj_param_obj.resolution
#     # print("-> resolution from projParam: %d" % proj_param_res)
#     return ulc_som, lrc_som

### raster resolution

In [6]:
# def setup_raster_resolution(ulc_lat_b1, ulc_lon_b1, lrc_lat_b1, lrc_lon_b1, img_ncols, img_nrows):
#     "calculate resolution for each block"
#     print("-> note: output res is in degrees.")
#     print("-> check inputs:")
#     print("ulc_lat_b1= %d" % ulc_lat_b1)
#     print("ulc_lon_b1= %d" % ulc_lon_b1)
#     print("lrc_lat_b1= %d" % lrc_lat_b1)
#     print("lrc_lon_b1= %d" % lrc_lon_b1)
    
#     #~ define resolution of each roughness img in degrees, based on science docs
#     cell_width_res = abs((lrc_lon_b1 - ulc_lon_b1) / float(img_ncols))
#     cell_height_res = abs((lrc_lat_b1 - ulc_lat_b1) / float(img_nrows))
#     print("-> W-res, H-res: %s, %s, in degrees" % (cell_width_res, cell_height_res))
#     return cell_width_res, cell_height_res

### extract corners by using Mtk-bls2latlon

In [7]:
def extract_block_corners_bls2latlon(path_num, misr_res_meter, block_num):    
    #~ in MISR view frame == North-down, ordered in clockwise manner from ulc
    #~ img pixels are 0-based

    img_ulc_dd = bls_to_latlon(path_num, misr_res_meter, block_num, 0, 0)       
    img_urc_dd = bls_to_latlon(path_num, misr_res_meter, block_num, 0, 2047)
    img_lrc_dd = bls_to_latlon(path_num, misr_res_meter, block_num, 511, 2047)
    img_llc_dd = bls_to_latlon(path_num, misr_res_meter, block_num, 511, 0)

    #~ note: from MISR perspective which is North-down
    print("extracted block corners for block %d:" % block_num)
    print("img_ulc: (%f, %f)" % img_ulc_dd)
    print("img_urc: (%f, %f)" % img_urc_dd) 
    print("img_lrc: (%f, %f)" % img_lrc_dd) 
    print("img_llc: (%f, %f)" % img_llc_dd)
    print("\n")
    
    return img_ulc_dd, img_urc_dd, img_lrc_dd, img_llc_dd

### raster-to-array --> North Down-Img ULC=origin ***

In [8]:
def array2raster_NorthDown_ImgULCAsOrigin(geotransform_rotated, ulc_lat, ulc_lon, width_pixel_res, height_pixel_res, nrows_img, ncols_img, raster_fullPath, roughness_array):
    "this function writes out the roughness image as a georeferenced tif file."
    "we use NorthUp notation in this function which assumes lookig at blocks in world crs/frame."

    print("-> using North-up (world CRS perspective) + img lrc as the origin of raster/block.")

    # reversed_arr = roughness_array[::-1] # reverse array so the tif looks like the array
    #~ define band and numeric type
    bands_num = 1
    datatype = gdal.GDT_Float32
    raster_epsg_code = 4326  # EPSG code for lat-lon # what is the ellipsoid information of this ellipsoid???
    x_rotation = 0 # 0 degrees if image is “North Up” , not supported!
    y_rotation = 0 # 0 degrees if image is “North Up” , not supported!
    
    print('-> input2raster: to write the raster we use MISR lrc as GDAL/world frame block origin (ulc)- lat,lon= %f, %f. ' % (ulc_lat, ulc_lon))

    #~ note: we define the area that will be covered to define the geo tranform; # units in degrees because our output raster is in lat-lon
    #~ parameter order: 
    #~ top left x (meters or degrees), w-e pixel resolution, rotation, top left y (meters or degrees), rotation, n-s pixel resolution; output units in map coords
    
    #geotransform_matrix = (lrc_lon, width_pixel_res, x_rotation, lrc_lat, y_rotation, -height_pixel_res) # note: negative flips the img over y-axis (in img coord. sys.), because we count from the UL corner to down

    #~ create output raster and setup driver=GTiff
    driver = gdal.GetDriverByName('GTiff')  # Initialize driver
    out_raster = driver.Create(raster_fullPath, ncols_img, nrows_img, bands_num, datatype)  # create output raster (img+georefrenced info) dataseet to write data into it (raster_fullPath, ncols_rough_arr, nrows_rough_arr, bands, dtype-> GDAL data type arg)
    out_raster.SetGeoTransform(geotransform_rotated)  # Set the affine transformation coefficients == Specify raster coordinates

    outband = out_raster.GetRasterBand(bands_num)
    outband.WriteArray(roughness_array)

    crs = osr.SpatialReference()
    crs.ImportFromEPSG(raster_epsg_code)
    # print("-> outout frame: %s" % crs.ExportToWkt())
    out_raster.SetProjection(crs.ExportToWkt())
    outband.FlushCache()

    #~ Once we're done, close the dataset properly
    out_raster = None
    outband = None
    print("-> FINISHED SUCCESS!")
    print(raster_fullPath)
    return 0





### setup pixel size 

In [9]:
# rasterx = 2048 # img W , img_ncols
# rastery = 512  # img H , img_nrows

# def setup_pixel_size_new(ulc, lrc, img_ncols, img_nrows):
#     #~ pixel size 
#     #x-component of the pixel width (x scale)
#     x_size = (ulc[1]-lrc[1])/img_ncols  # pixel-x-size; rasterx, ppx
#     # y-component of the pixel height (y scale)
#     y_size = (ulc[0]-lrc[0])/img_nrows  # pixel-y-size, rastery, ppy

#     print("-> each pixel size x&y (lon&lat) in degrees:")
#     print(x_size, y_size)  # W-res, H-res --> lon-lat
    
#     return x_size, y_size


def setup_pixel_size_new(img_ulc_dd, img_lrc_dd, misr_img_res):
    #x-component of the pixel width (x scale)
    pixel_w = abs((img_ulc_dd[1]-img_lrc_dd[1])/misr_img_res[1])  # pixel-x-size; rasterx, ppx
    # y-component of the pixel height (y scale)
    pixel_h = abs((img_ulc_dd[0]-img_lrc_dd[0])/misr_img_res[0])  # pixel-y-size, rastery, ppy
    #~ create a tuple
    pixel_size = (pixel_w, pixel_h)

    print("-> pixel size (w,h) in degrees:")
    print (pixel_size) # W-res, H-res --> lon-lat
    
    return pixel_size


# -> W-res, H-res: 0.00622356965449, 0.000917900200359, each pixel x&y size in degrees; 


### geotrnsform matrix --> with gdal and the 4 corners points as ground control points (GCP)

In [10]:
# def perspective_transformation_original(img_nrows, img_ncols, ulc, urc, lrc, llc, pixel_size):
#     "we use this function with North-up notation to define 4 corners of a block"
#     "usage: we use 4 corners as GCPs (ground control points) to define an area in geographic CRS"
#     "note: inut parameters are in img frame starting from MISR ulc"
    
#     # geotransform_rotated = perspective_transformation(img_ncols, img_nrows, llc, lrc, urc, ulc)  

#     print("\n -> computing Perspective Transformation:")
#     print("-> order of parameters input to geoTrans:")
#     print(img_nrows, img_ncols, ulc, urc, lrc, llc, pixel_size)
    
#     #~ img frame
#     fp = [[0, img_ncols, img_ncols, 0],     # img columns; x-dir
#           [0, 0, img_nrows, img_nrows]]      # img rows; y-dir

#     #~ world frame
#     tp = [[llc[1], lrc[1], urc[1],ulc[1]],         # lon
#           [llc[0], lrc[0], urc[0],ulc[0]]]          # lat

#     pix = list(zip(fp[0],fp[1])) # pixel corners
#     print("-> pix:")
#     for i in pix:
#         print(i)

#     coor = list(zip(tp[0],tp[1])) # corner coordinates
#     print("-> coor:")
#     for j in coor:
#         print(j)

#     # compute the gdal.GCP parameters
#     gcps = []  # ground control points
#     for index, txt in enumerate(pix):
#         #~ adding GCPs to a list
#         gcps.append(gdal.GCP())
#         gcps[index].GCPPixel = pix[index][0]
#         #gcps[index].GCPLine = rastery-int(pix[index][1])
#         gcps[index].GCPLine = pixel_size[1]-int(pix[index][1])

#         gcps[index].GCPX = coor[index][0]
#         gcps[index].GCPY = coor[index][1]

#     geotransform_rotated = gdal.GCPsToGeoTransform(gcps)
#     print("-> geotransform_rotated matrix: \n top-left X, w-e pixel size, rotation, top-left Y, rotation, n-s pixel size; output units in map coords")
#     print(geotransform_rotated)
# #     geotransform_rotated_2 = geotransform_rotated[:-1]*-1
# #     print("-> now" % geotransform_rotated_2)
#     print("\n")
    
#     return geotransform_rotated

In [11]:
def geoT_Ndown(img_nrows, img_ncols, ulc, urc, lrc, llc, pixel_size):  
    "we use this function with North-up notation to define 4 corners of a block"
    "usage: we use 4 corners as GCPs (ground control points) to define an area in geographic CRS"
    "note: inut parameters are in img frame starting from MISR ulc"
    "reference: https://gdal.org/user/raster_data_model.html#gcps"
    #~ to do h_reverese make it on==1
    h_reverse=0

    print("\n -> computing Perspective Transformation:")
    print("-> order of parameters input to geoTrans:")
    print(img_nrows, img_ncols, ulc, urc, lrc, llc, pixel_size)
    
    #~ img frame, NorthDown & from ulc+clockwise
    fp = [[0,  img_ncols, img_ncols,          0],     # img columns; x-dir
           [0,         0, img_nrows,  img_nrows]]      # img rows; y-dir
    

    #~ geographic/world frame, NorthDown ulc+clockwise -> note: these locations are 4 raster corners 
#     tp = [[llc[1], lrc[1], urc[1], ulc[1]],         # lon
#           [llc[0], lrc[0], urc[0], ulc[0]]]          # lat

    tp = [[ulc[1], urc[1], lrc[1], llc[1]],         # lon
          [ulc[0], urc[0], lrc[0], llc[0]]]          # lat

    #~ pixel corners
    pixel_coord = list(zip(fp[0],fp[1])) 
    print("-> pix:")
    for i in pixel_coord:
        print(i)

    #~ geographic coordinates
    coor = list(zip(tp[0],tp[1])) 
    print("-> coor:")
    for j in coor:
        print(j)

    # compute the gdal.GCP parameters
    gcp_list = []  # ground control points
    
    for index, txt in enumerate(pixel_coord):
        
        #~ adding GCPs to a list
        gcp_list.append(gdal.GCP())  # initialize GCP dataStruct
        gcp_list[index].GCPPixel = pixel_coord[index][0] # row
        #gcp_list[index].GCPLine = pixel_size[0]-int(pixel_coord[index][1]) # i modified; column
        gcp_list[index].GCPLine = pixel_coord[index][1] # i modified here; column
        print("GCP pixel&line: %d, %d" % (pixel_coord[index][0], pixel_coord[index][1]))
        #gcp_list[index].GCPLine = rastery     -int(pix[index][1]) # old
        
        gcp_list[index].GCPX = coor[index][0]
        gcp_list[index].GCPY = coor[index][1]
        print("GCP X,Y: %f, %f" % (coor[index][0], coor[index][1]))

    geoT_rotated = gdal.GCPsToGeoTransform(gcp_list)
    
    print("-> geoT_rotated: \n top-left X, w-e pixel size, rotation, top-left Y, rotation, n-s pixel size; output units in map coords")
    print(geoT_rotated)
    print("\n")

#     if (h_reverse):
#         print("-> h_reverse is on...")
#         #~ unpack tuple to reverse h value
#         (lon_ulc, w_pix, rotX, lon, rotY, h_pix) = geotransform_rotated
#         h_pix_reversed = h_pix*-1

#         #~ now pack everything back to a tuple
#         geotransform_rotated_Hreversed = (lon_ulc, w_pix, rotX, lon, rotY, h_pix_reversed)
#         print(geotransform_rotated_Hreversed)
        
#         return geoT_rotated_Hreversed
#         print("\n")
        
#     else:
    return geoT_rotated

### raster-to-array North-up+lrc as origin (world crs)

In [12]:
def arr2ras_Nup_ImgLRC(geotransform_rotated, width_pixel_res, height_pixel_res, nrows_img, ncols_img, raster_fullPath, roughness_array):
    "this function writes out the roughness image as a georeferenced tif file."
    "we use NorthUp notation in this function which assumes lookig at blocks in world crs/frame."

    print("-> using North-up (world CRS perspective) + img lrc as the origin of raster/block.")

    # reversed_arr = roughness_array[::-1] # reverse array so the tif looks like the array
    #~ define band and numeric type
    bands_num = 1
    datatype = gdal.GDT_Float32
    raster_epsg_code = 4326  # EPSG code for lat-lon # what is the ellipsoid information of this ellipsoid???
    x_rotation = 0 # 0 degrees if image is “North Up” , not supported!
    y_rotation = 0 # 0 degrees if image is “North Up” , not supported!
    
#     print('-> input2raster: to write the raster we use MISR lrc as GDAL/world frame block origin (ulc)- lat,lon= %f, %f' % (lrc_lat, lrc_lon))

    #~ note: we define the area that will be covered to define the geo tranform; # units in degrees because our output raster is in lat-lon
    #~ parameter order: 
    #~ top left x (meters or degrees), w-e pixel resolution, rotation, top left y (meters or degrees), rotation, n-s pixel resolution; output units in map coords
    
    #geotransform_matrix = (lrc_lon, width_pixel_res, x_rotation, lrc_lat, y_rotation, -height_pixel_res) # note: negative flips the img over y-axis (in img coord. sys.), because we count from the UL corner to down

    #~ create output raster and setup driver=GTiff
    driver = gdal.GetDriverByName('GTiff')  # Initialize driver
    out_raster = driver.Create(raster_fullPath, ncols_img, nrows_img, bands_num, datatype)  # create output raster (img+georefrenced info) dataseet to write data into it (raster_fullPath, ncols_rough_arr, nrows_rough_arr, bands, dtype-> GDAL data type arg)
    out_raster.SetGeoTransform(geotransform_rotated)  # Set the affine transformation coefficients == Specify raster coordinates

    outband = out_raster.GetRasterBand(bands_num)
    outband.WriteArray(roughness_array)

    crs = osr.SpatialReference()
    crs.ImportFromEPSG(raster_epsg_code)
    # print("-> outout frame: %s" % crs.ExportToWkt())
    out_raster.SetProjection(crs.ExportToWkt())
    outband.FlushCache()

    #~ Once we're done, close the dataset properly
    out_raster = None
    outband = None
    print("-> FINISHED SUCCESS!")
    print(raster_fullPath)
    return 0





# -----> main()

## check files and directories

In [13]:
root = "/Users/ehsanmos/Documents/RnD/Ehsan_lab_MISR/"

hdf_dir = os.path.join(root, "hdf_dir.nosync")  # Mtk projparam
# misr_dir = "/Volumes/Ehsanm_DRI/research/MISR/July_2016/July_01" # test for manual check of ProjParam
hdf_file = "MISR_AM1_GRP_ELLIPSOID_GM_P180_O088147_AN_F03_0024.hdf" # projParam from Mtk
# misr_file = "MISR_AM1_GRP_ELLIPSOID_GM_P233_O087961_CF_F03_0024.hdf" # for manual projParam
misr_file_fulPath = os.path.join(hdf_dir, hdf_file)

file_fullpath_checker(misr_file_fulPath)
print(misr_file_fulPath)

roughness_dir = "roughness_dir"
roughness_dir = os.path.join(root, roughness_dir)

#~ roughness files
roughness_file_b1 = "roughness_P180_O088147_B001.dat"
roughness_file_b2 = "roughness_P180_O088147_B002.dat"

roughness_file_b1_fullPath = os.path.join(roughness_dir, roughness_file_b1)
roughness_file_b2_fullPath = os.path.join(roughness_dir, roughness_file_b2)

#~ chech file path
# print(roughness_file_fullPath.split("/")[-1])
file_fullpath_checker(roughness_file_b1_fullPath)
print(roughness_file_b1_fullPath)

file_fullpath_checker(roughness_file_b2_fullPath)
print(roughness_file_b2_fullPath)

# print(roughness_file_fullPath)

#~ define raster directory
raster_dir = "raster_dir.nosync"
raster_dir_fullpath = os.path.join(root, raster_dir)
if not (os.path.isdir(raster_dir_fullpath)):
    print("-> raster dir NOT available!")
else:
    print(raster_dir_fullpath)


-> file available!
/Users/ehsanmos/Documents/RnD/Ehsan_lab_MISR/hdf_dir.nosync/MISR_AM1_GRP_ELLIPSOID_GM_P180_O088147_AN_F03_0024.hdf
-> file available!
/Users/ehsanmos/Documents/RnD/Ehsan_lab_MISR/roughness_dir/roughness_P180_O088147_B001.dat
-> file available!
/Users/ehsanmos/Documents/RnD/Ehsan_lab_MISR/roughness_dir/roughness_P180_O088147_B002.dat
/Users/ehsanmos/Documents/RnD/Ehsan_lab_MISR/raster_dir.nosync


## using Mtk "bls_to_latlon" to get corners

In [14]:
# for blk in range(1,2,1):
# print(Mtk.bls_to_latlon(180, 275, 1, 0, 0))
# print(Mtk.bls_to_latlon(180, 275, 1, 511, 2047))
# print("\n")
# print(Mtk.bls_to_latlon(180, 275, 2, 0, 0))
# print(Mtk.bls_to_latlon(180, 275, 2, 511, 2047))


## using Mtk "path_block_range_to_block_corners" to get corners

In [15]:
# block_corner_range_obj_dd = Mtk.MtkGeoBlock.llc.start_block.   #path_block_range_to_block_corners(180, 1, 1) #  (path, start block, end block)
# # b1 = 
# block_corner_range_obj_dd.start_block
# # len(b1)
# b1


# ulc = b1.ulc
# print("ulc=", ulc)
# llc = b1.llc
# print("llc=", llc)
# lrc = b1.lrc
# print("lrc=", lrc)
# urc = b1.urc
# print("urc=", urc)



# print("b-1 corner info:")
# print(b1.ulc)
# print(b1.lrc)

# print("\n")
# b2 = block_range_obj.block[2]
# print(b2.ulc)
# print(b2.lrc)

### extract corners by using Mtk-gridmethod (old)

In [16]:
# def extract_block_corners_gridmethod(path_num, block_num, misr_res_meter):
#     "we extract block corners by using MtkRegion(), order does not matter, but is in image frame= ulc,..."
#     "we use NorthDown perspetive in this function witch is the MISR camera perspective"
#     map_info = Mtk.MtkRegion(path_num, block_num, block_num).snap_to_grid(path_num, misr_res_meter)
#     lon_list = map_info.create_latlon()[0]
#     lat_list = map_info.create_latlon()[1]
# #     print(lon_list.shape)

#     #~ block corners, (lon, list)
#     print("-> these coords are in North-Down perspetive (along MISR ground track):")
#     #~ ulc
#     ulc = (lon_list[0,0], lat_list[0,0])
#     print("-> ulc= %f, %f" % (ulc))

#     #~ llc
#     llc = (lon_list[511, 0], lat_list[511, 0])
#     print("-> llc= %f, %f" % (llc))

#     #~ lrc
#     lrc = (lon_list[511, 2047], lat_list[511, 2047])
#     print("-> lrc= %f, %f" % (lrc))

#     #~ urc
#     urc = (lon_list[0,2047], lat_list[0,2047])
#     print("-> urc= %f, %f" % (urc))
    
    
#     return ulc, urc, lrc, llc

#     # latlon_b1_ulc = map_info.ls_to_latlon(0,0)
#     # print(latlon_b1_ulc)
#     # latlon_b1_lrc = map_info.ls_to_latlon(511,2047)
#     # print(latlon_b1_lrc)

## read roughness file and make roughness img 
and to get corners from previous research files
This is still in image coordinates

In [17]:
rough_file_b1 = np.fromfile(roughness_file_b1_fullPath, dtype='float64')     # is this roughness in cm?
rough_file_b2 = np.fromfile(roughness_file_b2_fullPath, dtype='float64')

# print("-> roughness file elements: %d" % rough_file.size)
# print("-> roughness file dim: %d" % rough_file.ndim)
# print("-> roughness file shape: %s" % rough_file.shape)


#~ get only the roughness values from file, roughness file has roughness-lat-lon in it as a list
rough_list_b1 = rough_file_b1[0:1048576]
rough_list_b2 = rough_file_b2[0:1048576]

# lat_arr = rough_file[1048576:2097152]  # this is from MISR2Roughness.c code and from MtkBlsToLatLon().
# lon_arr = rough_file[2097152:3145728]
# print("-> directly from input array:")
# print("-> 1st lat element of list: %s" % rough_file[1048576])
# print("-> last lat element of list: %s" % rough_file[2097151])

# print("-> shape of rough_arr: %d" % rough_arr.shape)
# print("-> shape of lat_arr: %d" % lat_arr.shape)
# print("-> shape of lon_arr: %d" % lon_arr.shape)

#~ create 2D roughness array from list
rough_img_b1 = rough_list_b1.reshape((512,2048))
rough_img_b2 = rough_list_b2.reshape((512,2048))

#~ increase img intensity by 10 to test things
rough_img_b1 = rough_img_b1*10
rough_img_b2 = rough_img_b2*10
print("-> roughness img-b1 new shape: %d,%d" % rough_img_b1.shape)
print("-> roughness img-b2 new shape: %d,%d" % rough_img_b2.shape)

# lat_img = lat_arr.reshape((512,2048))
# lon_img = lon_arr.reshape((512,2048))


# print("-> rough arr reshaped: %s" % type(roughness_img))
# print("-> roughness file elements: %d" % roughness_img.size)
# print("-> roughness file new dim: %d" % roughness_img.ndim)
# print("-> roughness file new shape: %d,%d" % roughness_img.shape)

# print(lat_img[0,0], lon_img[0,0])
# print(lat_img[-1,-1], lon_img[-1,-1])

-> roughness img-b1 new shape: 512,2048
-> roughness img-b2 new shape: 512,2048


NOTE: so far we treid 3 diff ways to get 1st block corners and all 3 were the same.

## get ProjParam from Mtk

In [18]:
# proj_param_obj = Mtk.MtkProjParam(misr_file_fulPath)
# print(proj_param_obj.projparam)

# offset_list = proj_param_obj.reloffset
# # for j in offset_list:
# #     print(j)

# print("-> number of pixels in offset list: %d" % len(offset_list))
# # print(len(offset_list))
# # print(sum(offset_list))

## process hdf file for b1 ulc, lrc, projParam, offset tuple

In [19]:
print("-> file is: %s" % hdf_file)
proj_param_obj = Mtk.MtkProjParam(misr_file_fulPath)

# print("-> ulc: (%d,%d) meters" % proj_param_obj.ulc)

print("-> projection parameter code: %s" % proj_param_obj.projcode)
b1_ulc_som = proj_param_obj.ulc  # order: (X,Y) = (lon,lat)
b1_lrc_som  = proj_param_obj.lrc  # order: (X,Y) = (lon,lat)

# # ground truth for reference
# print("-> ulc SOM from hdf file: (%d,%d) meters" % b1_ulc_som)
# print("-> lrc SOM from hdf file: (%d,%d) meters" % b1_lrc_som)

# swap y-coords
# b1_ulc_som_swaped, b1_lrc_som_swaped = swap_ul_lr_y_coord(b1_ulc_som, b1_lrc_som)

#~ get proj res
proj_param_res = proj_param_obj.resolution
print("-> resolution from projParam: %d" % proj_param_res)

#~ get offset values
offset = proj_param_obj.reloffset
print("-> number of offset pixels: %s" % len(offset))

print("-> offset/#pixels of origin/ulc of each block relative to the origin of its top block:")
print(offset)

-> file is: MISR_AM1_GRP_ELLIPSOID_GM_P180_O088147_AN_F03_0024.hdf
-> projection parameter code: 22
-> resolution from projParam: 1100
-> number of offset pixels: 179
-> offset/#pixels of origin/ulc of each block relative to the origin of its top block:
(0.0, 16.0, 0.0, 16.0, 0.0, 0.0, 0.0, 16.0, 0.0, 0.0, 0.0, 0.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -16.0, 0.0, 0.0, 0.0, -16.0, 0.0, 0.0, -16.0, 0.0, 0.0, -16.0, 0.0, -16.0, 0.0, -16.0, 0.0, -16.0, -16.0, 0.0, -16.0, 0.0, -16.0, -16.0, 0.0, -16.0, -16.0, -16.0, 0.0, -16.0, -16.0, -16.0, -16.0, 0.0, -16.0, -16.0, -16.0, -16.0, -16.0, -16.0, -16.0, -16.0, -16.0, -16.0, -16.0, -16.0, -16.0, -16.0, -16.0, -16.0, -16.0, -16.0, -16.0, -16.0, -16.0, -16.0, -16.0, -16.0, -16.0, -32.0, -16.0, -16.0, -16.0, -16.0, -16.0, -16.0, -16.0, -16.0, -16.0, -16.0, -32.0, -16.0, -16.0, -16.0, -16.0, -16.0, -16.0, -16.0, -16.0, -16.0, -16.0, -16.0, -16.0, -16.0, -16.0, -16.0, -16.0, -16.0, -16.0, -16.0, -16.0, -16.0, -16.

## change b1 Mtk.somxy_to_latlon

In [20]:
# # print(b1_ulc_som_swaped)
# # print(b1_lrc_som_swaped)

# b1_ulc_latlon = Mtk.somxy_to_latlon(path_num, b1_ulc_som_swaped[0], b1_ulc_som_swaped[1])  # order matters => to lat-lon of only ulc
# b1_lrc_latlon = Mtk.somxy_to_latlon(path_num, b1_lrc_som_swaped[0], b1_lrc_som_swaped[1])  # order matters

# print(b1_ulc_latlon)
# print(b1_lrc_latlon)
# #~ swap y-coords
# # b1_ulc_latlon_swaped, b1_lrc_latlon_swaped = swap_ul_lr_y_coord(b1_ulc_latlon, b1_lrc_latlon)

# # x_res = abs((b1_lr_lon-b1_ul_lon)/float(ncols_img))
# # y_res = abs((b1_lr_lat-b1_ul_lat)/float(nrows_img))
# # print("-> x_res: %s, y_res: %s" % (x_res,y_res))

## doing an example from Mtk src code

In [21]:
# example_latlon = Mtk.somxy_to_latlon(1, 10529200.016621, 622600.018066)  # order matters => to lat-lon of only ulc
# example_latlon

## set up resolution

In [22]:
# width_pixel_res, height_pixel_res = setup_raster_resolution(b1_ulc_latlon[0], b1_ulc_latlon[1], b1_lrc_latlon[0], b1_lrc_latlon[1], img_ncols, img_nrows)

# pixel_x_size, pixel_y_size = setup_pixel_res_new(ulc, lrc, img_ncols, img_nrows)

## writing block-1

In [23]:
# array2raster(b1_ulc_latlon[0], b1_ulc_latlon[1], width_pixel_res, height_pixel_res, img_ncols, img_nrows, raster_file_fullpath, rough_img_b1)

In [24]:
# for block_num in range(1,2,1):
    
#     print("-> processing block %d" % block_num)
    
#     #~ setup block-1 raster fullpath
#     raster_name = "path_"+str(path_num)+"_block_"+str(block_num)
#     raster_file_fullpath = setup_rastername(raster_name, raster_dir_fullpath)
    
#     ulc, urc, lrc, llc = extract_block_corners(path_num, block_num, misr_res_meter)
    
#     geotransform_rotated = perspectiveTransform(img_ncols, img_nrows, llc, lrc, urc, ulc)
    
#     #~ old one 
#     # array2raster_NorthUp_LRCOrigin(geotransform_rotated, b1_lrc_latlon[0], b1_lrc_latlon[1], width_pixel_res, height_pixel_res, img_ncols, img_nrows, raster_file_fullpath, rough_img_b1)

#     array2raster_NorthUp_LRCOrigin(geotransform_rotated, b1_lrc_latlon[0], b1_lrc_latlon[1], pixel_x_size, pixel_y_size, img_ncols, img_nrows, raster_file_fullpath, rough_img_b1)

## setup block-1 raster fullpath

In [25]:
# for block_num in range(1,2,1):

# #~ setup block-2 raster fullpath
# raster_name = "path_"+str(path_num)+"_block_"+str(block_num)
# raster_file_fullpath = setup_rastername(raster_name, raster_dir_fullpath)

## process block-1

In [26]:
block_num = 1
print("-> processing block %d" % block_num)

raster_name = "path_"+str(path_num)+"_block_"+str(block_num)
raster_file_fullpath = setup_rastername(raster_name, raster_dir_fullpath)

#~ extract block corners
#~ how extract corners better/faster?
img_ulc_dd, img_urc_dd, img_lrc_dd, img_llc_dd = extract_block_corners_bls2latlon(path_num, misr_res_meter, block_num)
     
#~ setup pixelsize
pixel_size = setup_pixel_size_new(img_ulc_dd, img_lrc_dd, misr_img_res)

#~ setup geoTransformation matrix
geoT_rotated = geoT_Ndown(misr_img_res[0], misr_img_res[1], img_ulc_dd, img_urc_dd, img_lrc_dd, img_llc_dd, pixel_size) 

#~ write img as ratser
#~ use img ulc as raster corner with NorthDown notation
# array2raster_NorthDown_ImgULCAsOrigin(geoT_rotated, img_ulc_dd[0], img_ulc_dd[1], pixel_size[0], pixel_size[1], misr_img_res[0], misr_img_res[1], raster_file_fullpath, rough_img_b1)

arr2ras_Nup_ImgLRC(geoT_rotated, pixel_size[0], pixel_size[1], misr_img_res[0], misr_img_res[1], raster_file_fullpath, rough_img_b1)

    
    
    

#~ old ones
# array2raster_NorthUp_LRCOrigin(geotransform_rotated, b1_lrc_latlon[0], b1_lrc_latlon[1], width_pixel_res, height_pixel_res, img_ncols, img_nrows, raster_file_fullpath, rough_img_b1)

# array2raster_NorthUp_LRCOrigin(geotransform_rotated, b1_lrc_latlon[0], b1_lrc_latlon[1], pixel_x_size, pixel_y_size, img_ncols, img_nrows, raster_file_fullpath, rough_img_b1)

#~ note: we use img lrc as raster origin here because we are using NorthUp notation and img lrc is the real origin of the block from sensor perspective in NorthDown notation
# array2raster_NorthUp_LRCOrigin(geotransform_rotated, img_lrc_dd[0], img_lrc_dd[1], pixel_size[0], pixel_size[1], misr_img_res[0], misr_img_res[1], raster_file_fullpath, rough_img_b1)


-> processing block 1
-> output raster: path_180_block_1_LatLon_degreeRes_NorthUp_perspectiveTrans.tif
/Users/ehsanmos/Documents/RnD/Ehsan_lab_MISR/raster_dir.nosync/path_180_block_1_LatLon_degreeRes_NorthUp_perspectiveTrans.tif
extracted block corners for block 1:
img_ulc: (66.223581, -166.103258)
img_urc: (64.605643, -177.602425)
img_lrc: (65.752107, -178.841471)
img_llc: (67.446689, -166.860177)


-> pixel size (w,h) in degrees:
(0.006219830389738862, 0.0009208472018909508)

 -> computing Perspective Transformation:
-> order of parameters input to geoTrans:
(512, 2048, (66.22358078624517, -166.1032584351828), (64.60564343453649, -177.6024248300928), (65.752107018877, -178.84147107336798), (67.44668851679545, -166.86017720787214), (0.006219830389738862, 0.0009208472018909508))
-> pix:
(0, 0)
(2048, 0)
(2048, 512)
(0, 512)
-> coor:
(-166.1032584351828, 66.22358078624517)
(-177.6024248300928, 64.60564343453649)
(-178.84147107336798, 65.752107018877)
(-166.86017720787214, 67.44668851679

0

In [27]:
block_num = 2
print("-> processing block %d" % block_num)

raster_name = "path_"+str(path_num)+"_block_"+str(block_num)
raster_file_fullpath = setup_rastername(raster_name, raster_dir_fullpath)

#~ extract block corners
#~ how extract corners better/faster?
img_ulc_dd, img_urc_dd, img_lrc_dd, img_llc_dd = extract_block_corners_bls2latlon(path_num, misr_res_meter, block_num)
     
#~ setup pixelsize
pixel_size = setup_pixel_size_new(img_ulc_dd, img_lrc_dd, misr_img_res)

#~ setup geoTransformation matrix
geoT_rotated = geoT_Ndown(misr_img_res[0], misr_img_res[1], img_ulc_dd, img_urc_dd, img_lrc_dd, img_llc_dd, pixel_size) 

#~ write img as ratser
#~ use img ulc as raster corner with NorthDown notation
# array2raster_NorthDown_ImgULCAsOrigin(geoT_rotated, img_ulc_dd[0], img_ulc_dd[1], pixel_size[0], pixel_size[1], misr_img_res[0], misr_img_res[1], raster_file_fullpath, rough_img_b1)

arr2ras_Nup_ImgLRC(geoT_rotated, pixel_size[0], pixel_size[1], misr_img_res[0], misr_img_res[1], raster_file_fullpath, rough_img_b2)

    

-> processing block 2
-> output raster: path_180_block_2_LatLon_degreeRes_NorthUp_perspectiveTrans.tif
/Users/ehsanmos/Documents/RnD/Ehsan_lab_MISR/raster_dir.nosync/path_180_block_2_LatLon_degreeRes_NorthUp_perspectiveTrans.tif
extracted block corners for block 2:
img_ulc: (67.449078, -166.861731)
img_urc: (65.754340, -178.844006)
img_lrc: (66.889158, 179.799413)
img_llc: (68.668316, -167.697312)


-> pixel size (w,h) in degrees:
(0.16926813680681696, 0.0010935941327590015)

 -> computing Perspective Transformation:
-> order of parameters input to geoTrans:
(512, 2048, (67.44907847403444, -166.86173134299995), (65.7543397184382, -178.84400581519114), (66.88915827806183, 179.7994128373612), (68.66831570229306, -167.69731185865032), (0.16926813680681696, 0.0010935941327590015))
-> pix:
(0, 0)
(2048, 0)
(2048, 512)
(0, 512)
-> coor:
(-166.86173134299995, 67.44907847403444)
(-178.84400581519114, 65.7543397184382)
(179.7994128373612, 66.88915827806183)
(-167.69731185865032, 68.668315702293

0